In [ ]:
import numpy as np
import pandas as pd
import talib
import yfinance as yf
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# 下载数据
stock_code = "AAPL"
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=10)
data = yf.download(stock_code, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

# 特征工程
data['Returns'] = data['Close'].pct_change()
data['HL_PCT'] = (data['High'] - data['Low']) / data['Close'] * 100.0
data['PCT_change'] = (data['Close'] - data['Open']) / data['Open'] * 100.0

# 添加技术指标
data['RSI'] = talib.RSI(data['Close'], timeperiod=14)
data['ADX'] = talib.ADX(data['High'], data['Low'], data['Close'], timeperiod=14)
data['OBV'] = talib.OBV(data['Close'], data['Volume'])
data['MACD'], data['MACD_signal'], data['MACD_hist'] = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['ATR'] = talib.ATR(data['High'], data['Low'], data['Close'], timeperiod=14)
data['slowk'], data['slowd'] = talib.STOCH(data['High'], data['Low'], data['Close'], fastk_period=14, slowk_period=3, slowd_period=3)
data['CCI'] = talib.CCI(data['High'], data['Low'], data['Close'], timeperiod=14)
upperband, middleband, lowerband = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['upperband'] = upperband
data['middleband'] = middleband
data['lowerband'] = lowerband
data['MOM'] = talib.MOM(data['Close'], timeperiod=10)

# 预测标签
prediction_days = 7
data['Future_Close'] = data['Close'].shift(-prediction_days)
data['Target'] = (data['Future_Close'] > data['Close']).astype(int)
data = data.dropna()

# 准备数据集
feature_columns = [col for col in data.columns if col not in ['Close', 'Future_Close', 'Target']]
X = data[feature_columns]
y = data['Target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 定义随机森林模型和扩展的超参数网格
model = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'bootstrap': [True, False]
}

# 使用GridSearchCV进行超参数优化
grid = GridSearchCV(model, param_grid, refit=True, verbose=2, cv=5)
grid.fit(X_train, y_train)

# 输出最佳参数
print("Best Parameters:", grid.best_params_)

# 使用最佳参数进行预测
best_model = grid.best_estimator_
predictions = best_model.predict(X_test)
prediction_proba = best_model.predict_proba(X_test)[:, 1]
report = classification_report(y_test, predictions)
fpr, tpr, _ = roc_curve(y_test, prediction_proba)
roc_auc = roc_auc_score(y_test, prediction_proba)

# 打印评估指标
print("Classification Report:")
print(report)
print("ROC AUC Score:", roc_auc)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [2]:
import tensorflow as tf
tf.__version__

c:\Users\48869\anaconda3\envs\tf_gpu\lib\site-packages\tensorflow\python\keras\engine\training_arrays_v1.py:37: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.19.5)
  from scipy.sparse import issparse  # pylint: disable=g-import-not-at-top


'2.5.3'

In [1]:
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 13:30:38) [MSC v.1916 64 bit (AMD64)]


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 设置GPU的内存增长
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # 设置可见的GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 设置GPU的内存增长
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # 设置可见的GPU
        tf.config.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1
